In [ ]:
import numpy as np
import numpy.linalg as npla
import scipy as sp
import scipy.signal as sps

import sys
import os
import glob
import pickle as pkl

import matplotlib.pyplot as plt

from aopy import datafilter, datareader

# modules that aren't done yet
sys.path.append('/Users/mickey/aoLab/code/analyze/')
# sys.path.append('/home/mickey/analyze/')
from tfspec import tfspec

In [ ]:
# spectrogram creation + saving loop function
def create_and_save_spectrogram(ecog_file, window_t, step_t, bw):
    ## create filename for sgram, mask files
    file_dir = os.path.dirname(ecog_file)
    file_base = os.path.basename(ecog_file)
    file_kern, file_ext = os.path.splitext(file_base)
    sgram_file_base = file_kern + 'sgram'
    sgram_mask_file_base = file_kern + 'sgram.mask.pkl'
    sgram_mask_file = os.path.join(file_dir,sgram_mask_file_base)
    
    print('Processing spectrogram for file {}:'.format(ecog_file))
    
    ## load data
    data, exp, mask = datareader.load_ecog_clfp_data(ecog_file)
    srate = exp['srate']
    n_ch = exp['num_ch']
#     data[:,mask['hf'] & mask['sat']] = 0.
    
    ## create spectrogram with tfspec
    # create taper structure
    window_n = window_t*srate
    step_n = step_t*srate
    nw = window_t*bw/2
    n_taper = 2*nw - 1
    tapers = [window_t, nw, n_taper]
    # compute sgram (all channels, all time - might be too much memory)
    sgram_all = []
    n_freq_bin = int(2**(np.ceil(np.log2(srate*window_t))))
    n_window = int(np.floor((data.shape[-1]-srate*window_t)/(srate*step_t)))
    sgram = np.zeros((n_ch,n_freq_bin,n_window),dtype=np.float32)
    print('computing spectrograms...')
    for ch_idx in range(n_ch):
        print(f'{ch_idx+1}/{n_ch}')
        sgram_, f_sgram, tidx_sgram, _ = tfspec(data[ch_idx,:][None,:],tapers=tapers,sampling=srate,dn=step_t)
        breakpoint()
        sgram[ch_idx,:,:] = np.float32(sgram_[0,:,:].copy().T)
        del sgram_

    ## create mask for noisy time regions of spectrogram
    print('creating data mask...')
    f_idx = f_sgram > 80.
    window_power = np.log10(sgram[:,:,f_idx]).mean(axis=-1)
    high_power_med = np.median(window_power,axis=-1)
    high_power_std = window_power.std(axis=-1)
    high_power_thresh = high_power_med + high_power_std/4
    high_power_win = high_power_thresh[:,None] < window_power
    breakpoint()

    ## reshape and save spectrogram with np.tofile()
    n_fft = len(f_sgram)
    n_win = len(tidx_sgram)
    sgram = sgram.reshape((n_ch*n_fft,n_win))
    print('Saving spectrogram (ch {}):'.format(ch_idx))
    sgram_file = os.path.join(file_dir,sgram_file_base) + file_ext
    print('saving spectrogram file...')
    sgram.tofile(sgram_file)

    ## save mask
    print('saving spectrogram mask...')
    with open(sgram_mask_file,'wb') as f:
        pkl.dump(high_power_win,f)
        
    return (sgram_file, sgram_mask_file)

In [ ]:
window_t = 4
step_t = 0.5
bw = 1
# get file list
file_dir = '/Volumes/Samsung_T5/aoLab/Data/WirelessData/Goose_Multiscale_M1/180325/0*/'
file_list = glob.glob(os.path.join(file_dir,'*.clfp_ds250.dat'))
# print(file_list)

for ecog_file in file_list[1:]:
    print(ecog_file)
    create_and_save_spectrogram(ecog_file,window_t,step_t,bw)

In [ ]:
# plt.plot(high_power_win)